<a href="https://colab.research.google.com/github/mterion/Building-with-Instruction-Tuned-LLMs-A-Step-by-Step-Guide/blob/main/3_Natural_Language_to_SQL_with_OpenLM_and_Lit_LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruct-tuning the Lit-LLaMA Model For SQL Task

In the following notebook, we'll be going through Instruct-tuning the Lit-LLaMA model using OpenLM Research [weights](https://huggingface.co/openlm-research/open_llama_7b_preview_200bt) trained using [RedPajama](https://www.together.xyz/blog/redpajama)'s dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/Lightning-AI/lit-llama
%cd lit-llama

Cloning into 'lit-llama'...
remote: Enumerating objects: 1556, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1556 (delta 643), reused 572 (delta 553), pack-reused 778
Receiving objects: 100% (1556/1556), 491.97 KiB | 27.33 MiB/s, done.
Resolving deltas: 100% (958/958), done.
/content/lit-llama


In [ ]:
!pip install -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 

In [ ]:
!git-lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
!git clone https://huggingface.co/openlm-research/open_llama_7b_400bt_preview checkpoints/open-llama/7B

Cloning into 'checkpoints/open-llama/7B'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), 3.41 KiB | 1.14 MiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 47.07 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/7B/ --model_size 7B

Initializing lit-llama
Processing pytorch_model-00002-of-00002.bin
Processing pytorch_model-00001-of-00002.bin
Saving to disk at checkpoints/lit-llama/7B


In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikisql")

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

Dataset wikisql downloaded and prepared to /root/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset["train"][0]

{'phase': 1,
 'question': 'Tell me what the notes are for South Australia ',
 'table': {'header': ['State/territory',
   'Text/background colour',
   'Format',
   'Current slogan',
   'Current series',
   'Notes'],
  'page_title': '',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-1000181-1',
  'section_title': '',
  'caption': '',
  'rows': [['Australian Capital Territory',
    'blue/white',
    'Yaa·nna',
    'ACT · CELEBRATION OF A CENTURY 2013',
    'YIL·00A',
    'Slogan screenprinted on plate'],
   ['New South Wales',
    'black/yellow',
    'aa·nn·aa',
    'NEW SOUTH WALES',
    'BX·99·HI',
    'No slogan on current series'],
   ['New South Wales',
    'black/white',
    'aaa·nna',
    'NSW',
    'CPX·12A',
    'Optional white slimline series'],
   ['Northern Territory',
    'ochre/white',
    'Ca·nn·aa',
    'NT · OUTBACK AUSTRALIA',
    'CB·06·ZZ',
    'New series began in June 2011'],
   ['Queensland',
    'maroon/white',
    'nnn·aaa

In [ ]:
json_list = []

for row in dataset["train"]:
  json_list.append({"instruction" : "Convert the following natural language question into SQL.", "input" : row["question"], "output" : row["sql"]["human_readable"]})

In [ ]:
import json

with open('data.json', 'w') as f:
    json.dump(json_list, f, indent=4)

In [ ]:
json_list[0]

{'instruction': 'Convert the following natural language question into SQL.',
 'input': 'Tell me what the notes are for South Australia ',
 'output': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}

In [ ]:
!python scripts/prepare_alpaca.py

train has 54,355 samples
val has 2,000 samples
Processing train split ...
100% 54355/54355 [00:30<00:00, 1786.68it/s]
Processing test split ...
100% 2000/2000 [00:01<00:00, 1699.54it/s]


### ⚠ IMPORTANT ⚠ 

In order to avoid size issues - you'll need to modify `max_seq_length = 1028` in the file `finetune/lora.py`!

In [ ]:
!python finetune/lora.py --data_dir data/alpaca --out_dir /content/drive/MyDrive/Eng-SQL/

Global seed set to 1337
iter 0: loss 2.8981, time: 1706.55ms
iter 1: loss 3.0892, time: 191.14ms
iter 2: loss 3.1028, time: 180.28ms
iter 3: loss 3.1511, time: 154.44ms
iter 4: loss 3.1369, time: 155.82ms
iter 5: loss 3.1258, time: 155.58ms
iter 6: loss 2.9861, time: 147.36ms
iter 7: loss 3.1632, time: 157.60ms
iter 8: loss 2.9969, time: 175.17ms
iter 9: loss 2.9357, time: 161.65ms
iter 10: loss 3.1545, time: 148.73ms
iter 11: loss 2.8026, time: 149.57ms
iter 12: loss 3.1507, time: 147.56ms
iter 13: loss 3.1744, time: 148.36ms
iter 14: loss 2.8080, time: 173.53ms
iter 15: loss 2.9459, time: 157.06ms
iter 16: loss 2.9195, time: 195.79ms
iter 17: loss 3.1160, time: 157.54ms
iter 18: loss 3.0989, time: 151.59ms
iter 19: loss 3.2050, time: 156.72ms
iter 20: loss 2.9236, time: 148.55ms
iter 21: loss 3.0760, time: 160.26ms
iter 22: loss 3.0785, time: 149.78ms
iter 23: loss 2.8673, time: 158.24ms
iter 24: loss 2.9882, time: 161.35ms
iter 25: loss 3.0642, time: 158.08ms
iter 26: loss 2.8223, t

In [ ]:
!python generate/lora.py --prompt "### INSTRUCTION\nConvert the following natural language question into SQL.\n\n### INPUT\nHow much wood could a woodchuck chuck, if a woodchuck could chuck wood?\n\n### OUTPUT\n" --lora_path "/content/drive/MyDrive/Eng-SQL/iter-003199-ckpt.pth"

Loading model ...
Time to load model: 21.22 seconds.
SELECT COUNT Woodchuck FROM table WHERE Woodchuck Chucks = Woodchuck


Time for inference: 0.87 sec total, 114.76 tokens/sec
Memory used: 27.65 GB


In [ ]:
!python generate/lora.py --prompt "### INSTRUCTION\nConvert the following natural language question into SQL.\n\n### INPUT\nOHow many students are between the ages of 10 and 30?\n\n### OUTPUT\n" --lora_path "/content/drive/MyDrive/Eng-SQL/iter-003199-ckpt.pth"

Loading model ...
Time to load model: 21.27 seconds.
SELECT COUNT Student FROM table WHERE Age < 40 AND Age > 10


Time for inference: 0.87 sec total, 114.59 tokens/sec
Memory used: 27.65 GB
